Importações

In [1]:
import graph_tool_extras as gte
from graph_tool import draw
import numpy as np
import pandas as pd
import csv
import netpixi

atribuindo path

In [2]:
PATH = './bilateral-remittance.csv'

## Leitura dos dados

In [3]:
data = pd.read_csv(PATH)

In [4]:
data.head()

,Unnamed: 0,Remittance-receiving country (across) - Remittance-sending country (down),Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Antigua and Barbuda,Argentina,...,Uzbekistan,Vanuatu,"Venezuela, RB",Vietnam,Virgin Islands (U.S.),West Bank and Gaza,"Yemen, Rep.",Zambia,Zimbabwe,World
0,0,Afghanistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,172.238651
1,1,Albania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,184.977059
2,2,Algeria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.00000,0.0,30.163722,7.459294,0.0,0.0,189.223264
3,3,American Samoa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.05771,0.0,0.000000,0.000000,0.0,0.0,32.184421
4,4,Andorra,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.403279,...,0.0,0.0,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.0,279.963568


## Construção do grafo

In [5]:
g = gte.Graph(directed=True) # dirigida



In [6]:
# Adicionar propriedades ao grafo
g.add_vp('total_out')  # Total de remessas enviadas por cada país
g.add_vp('total_in')   # Total de remessas recebidas por cada país
g.add_ep('value')      # Valor da remessa entre dois países

In [7]:
# Lista de países (colunas do dataset, excluindo a primeira que é apenas um índice)
# O primeiro problema está aqui - vamos corrigir a lista de países
# Primeiro, vamos ignorar a primeira coluna que é apenas um índice
data_columns = data.columns[1:]

# O primeiro nome de coluna parece ser o cabeçalho que contém a descrição 
# Vamos removê-lo da lista de países
header_column = data_columns[0]  # Coluna com o nome longo que causa o erro
actual_countries = data_columns[1:]  # Lista real de países (colunas)

# Vamos ver quais são os países de origem (nas linhas)
sending_countries = data[header_column].tolist()
sending_countries = [country for country in sending_countries if isinstance(country, str)]

# Criar um conjunto de todos os países (união dos que enviam e recebem)
all_countries = set(actual_countries).union(set(sending_countries))

In [8]:
# Adicionar países como vértices
for country in all_countries:
    if country and isinstance(country, str):  # Verificar se não é None ou NaN
        v = g.add_vertex_by_id(country)
        v['total_out'] = 0
        v['total_in'] = 0

In [9]:
# Adicionar remessas como arestas
for i, row in data.iterrows():
    sender = row[header_column]
    
    # Verificar se o remetente é válido
    if not isinstance(sender, str):
        continue
        
    for receiver in actual_countries:
        # Verificar se temos um valor válido de remessa
        try:
            # Tentar limpar o valor (remover caracteres não numéricos)
            value_str = str(row[receiver])
            # Remover '*' e outros caracteres não numéricos, mas manter o ponto decimal
            value_clean = ''.join(c for c in value_str if c.isdigit() or c == '.')
            
            # Se depois de limpar, ainda temos um valor
            if value_clean and pd.notna(value_clean):
                value = float(value_clean)
                if value > 0:
                    # Verificar se ambos os países existem no grafo
                    if g.vertex_by_id(sender) is not None and g.vertex_by_id(receiver) is not None:
                        # Adicionar aresta representando a remessa
                        e = g.add_edge_by_ids(sender, receiver)
                        e['value'] = value
                        
                        # Atualizar totais
                        sender_vertex = g.vertex_by_id(sender)
                        receiver_vertex = g.vertex_by_id(receiver)
                        
                        sender_vertex['total_out'] += value
                        receiver_vertex['total_in'] += value
        except (ValueError, TypeError):
            # Se houver algum erro de conversão, simplesmente pular este valor
            continue

In [10]:
# Salvar a rede
gte.save(g, 'remittances_network.net.gz')

In [11]:
# Info 
num_vertices = g.num_vertices()
num_edges = g.num_edges()

print(f"A rede de remessas tem {num_vertices} vértices (países)")
print(f"A rede de remessas tem {num_edges} arestas (fluxos de remessas)")

A rede de remessas tem 215 vértices (países)
A rede de remessas tem 12295 arestas (fluxos de remessas)


In [12]:
# Posicionar os vértices para visualização
layout = draw.sfdp_layout(g)
gte.move(g, layout)
gte.save(g, 'remittances_network_positioned.net.gz')

In [13]:
# Visualizar a rede
netpixi.render('remittances_network_positioned.net.gz')

In [14]:
# Coletar totais de remessas enviadas e recebidas por país
country_totals_sent = {}
country_totals_received = {}

for i, row in data.iterrows():
    sender = row[header_column]
    
    if not isinstance(sender, str):
        continue
    
    # Inicializar remetente se ainda não existir
    if sender not in country_totals_sent:
        country_totals_sent[sender] = 0
    
    for receiver in actual_countries:
        # Inicializar destinatário se ainda não existir
        if receiver not in country_totals_received:
            country_totals_received[receiver] = 0
        
        try:
            value_str = str(row[receiver])
            value_clean = ''.join(c for c in value_str if c.isdigit() or c == '.')
            
            if value_clean and pd.notna(value_clean):
                value = float(value_clean)
                if value > 0:
                    # Adicionar ao total do remetente
                    country_totals_sent[sender] += value
                    # Adicionar ao total do destinatário
                    country_totals_received[receiver] += value
        except (ValueError, TypeError):
            continue

# Converter para listas para calcular estatísticas
sent_values = list(country_totals_sent.values())
received_values = list(country_totals_received.values())

# Calcular medianas
if sent_values:
    median_sent = np.median(sent_values)
    print(f"Mediana de remessas enviadas: {median_sent:.2f}")
else:
    median_sent = 0
    print("Não há dados suficientes para calcular a mediana de remessas enviadas")

if received_values:
    median_received = np.median(received_values)
    print(f"Mediana de remessas recebidas: {median_received:.2f}")
else:
    median_received = 0
    print("Não há dados suficientes para calcular a mediana de remessas recebidas")

# Definir limites (por exemplo, 10% da mediana)
min_sent_threshold = median_sent * 0.1
min_received_threshold = median_received * 0.1

print(f"Limite mínimo para países remetentes: {min_sent_threshold:.2f}")
print(f"Limite mínimo para países destinatários: {min_received_threshold:.2f}")

# Filtrar países relevantes
relevant_senders = {country for country, total in country_totals_sent.items() if total >= min_sent_threshold}
relevant_receivers = {country for country, total in country_totals_received.items() if total >= min_received_threshold}

# União dos conjuntos para ter todos os países relevantes
relevant_countries = relevant_senders.union(relevant_receivers)

print(f"Número de países remetentes relevantes: {len(relevant_senders)}")
print(f"Número de países destinatários relevantes: {len(relevant_receivers)}")
print(f"Número total de países relevantes: {len(relevant_countries)}")

# Criar o grafo apenas com países relevantes
g = gte.Graph(directed=True)
g.add_vp('total_out')
g.add_vp('total_in')
g.add_ep('value')

# Adicionar apenas países relevantes como vértices
for country in relevant_countries:
    v = g.add_vertex_by_id(country)
    v['total_out'] = country_totals_sent.get(country, 0)
    v['total_in'] = country_totals_received.get(country, 0)

# Adicionar remessas como arestas (apenas entre países relevantes)
for i, row in data.iterrows():
    sender = row[header_column]
    
    if not isinstance(sender, str) or sender not in relevant_countries:
        continue
        
    for receiver in actual_countries:
        if receiver not in relevant_countries:
            continue
            
        try:
            value_str = str(row[receiver])
            value_clean = ''.join(c for c in value_str if c.isdigit() or c == '.')
            
            if value_clean and pd.notna(value_clean):
                value = float(value_clean)
                if value > 0:
                    e = g.add_edge_by_ids(sender, receiver)
                    e['value'] = value
        except (ValueError, TypeError):
            continue

# Imprimir informações sobre a rede filtrada
print(f"\nRede filtrada:")
print(f"- Número de vértices (países): {g.num_vertices()}")
print(f"- Número de arestas (fluxos de remessas): {g.num_edges()}") 

Mediana de remessas enviadas: 435.34
Mediana de remessas recebidas: 741.19
Limite mínimo para países remetentes: 43.53
Limite mínimo para países destinatários: 74.12
Número de países remetentes relevantes: 185
Número de países destinatários relevantes: 158
Número total de países relevantes: 199

Rede filtrada:
- Número de vértices (países): 199
- Número de arestas (fluxos de remessas): 11687


In [15]:
# Criar propriedades para reduzir tamanho dos vértices e espessura das arestas
vertex_size = g.new_vp("double")
edge_width = g.new_ep("double")

# Definir um tamanho fixo menor para os vértices
default_vertex_size = 5  # Valor arbitrário para simplificação

# Definir uma espessura fixa menor para as arestas
default_edge_width = 0.5  # Valor arbitrário para simplificação

# Aplicar os valores aos vértices e arestas
for v in g.vertices():
    vertex_size[v] = default_vertex_size

for e in g.edges():
    edge_width[e] = default_edge_width

print("\nAjustes aplicados:")
print(f"- Tamanho dos vértices reduzido para {default_vertex_size}")
print(f"- Espessura das arestas reduzida para {default_edge_width}")


Ajustes aplicados:
- Tamanho dos vértices reduzido para 5
- Espessura das arestas reduzida para 0.5


In [16]:
layout = draw.sfdp_layout(g)
gte.move(g, layout)
gte.save(g, 'remittances_network_positioned_2.net.gz')

In [17]:
netpixi.render('remittances_network_positioned_2.net.gz')

In [18]:
from graph_tool.centrality import closeness
import graph_tool.all as gte

# Calcular a centralidade de proximidade para cada vértice
closeness_vp = closeness(g)

# Extrair os valores de closeness para normalização
closeness_values = np.array([closeness_vp[v] for v in g.vertices()])

# Definir tamanho mínimo e máximo para os vértices (ajustáveis)
min_size = 5   # tamanho mínimo
max_size = 20  # tamanho máximo

min_closeness = closeness_values.min()
max_closeness = closeness_values.max()

# Criar uma propriedade de vértice para os tamanhos baseados em closeness
vertex_size_closeness = g.new_vp("double")
for v in g.vertices():
    closeness_val = closeness_vp[v]
    # Normalização linear dos valores para o intervalo [min_size, max_size]
    if max_closeness > min_closeness:
        normalized = (closeness_val - min_closeness) / (max_closeness - min_closeness)
    else:
        normalized = 0
    vertex_size_closeness[v] = min_size + normalized * (max_size - min_size)

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Gerar o diagrama, definindo o tamanho dos vértices conforme o closeness
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_closeness,
    output_size=(1000, 1000),
    output="graph_closeness.png"
)

print("Diagrama gerado: 'graph_closeness.png'")


Diagrama gerado: 'graph_closeness.png'


In [19]:
# Calcular o degree (grau) para cada vértice
# Em uma rede direcionada, podemos olhar para grau de entrada, saída ou total

# Criar uma propriedade de vértice para o grau
degree_prop = g.new_vp("int")

# Calcular o grau total (entrada + saída) para cada vértice
for v in g.vertices():
    # Em uma rede direcionada, o grau total é a soma do grau de entrada e saída
    in_deg = v.in_degree()
    out_deg = v.out_degree()
    degree_prop[v] = in_deg + out_deg

# Extrair os valores de degree para normalização
degree_values = np.array([degree_prop[v] for v in g.vertices()])

# Definir tamanho mínimo e máximo para os vértices
min_size = 5   # tamanho mínimo
max_size = 20  # tamanho máximo

min_degree = degree_values.min()
max_degree = degree_values.max()

# Criar uma propriedade de vértice para os tamanhos baseados no degree
vertex_size_degree = g.new_vp("double")
for v in g.vertices():
    degree_val = degree_prop[v]
    # Normalização linear dos valores para o intervalo [min_size, max_size]
    if max_degree > min_degree:
        normalized = (degree_val - min_degree) / (max_degree - min_degree)
    else:
        normalized = 0
    vertex_size_degree[v] = min_size + normalized * (max_size - min_size)

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Gerar o diagrama, definindo o tamanho dos vértices conforme o degree
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_degree,
    output_size=(1000, 1000),
    output="graph_degree.png"
)

print("Diagrama gerado: 'graph_degree.png'")

Diagrama gerado: 'graph_degree.png'


In [20]:
from graph_tool.centrality import betweenness

# Calcular a centralidade de intermediação (betweenness) para cada vértice
vertex_betweenness, edge_betweenness = betweenness(g)

# Extrair os valores de betweenness para normalização
betweenness_values = np.array([vertex_betweenness[v] for v in g.vertices()])

# Definir tamanho mínimo e máximo para os vértices
min_size = 5   # tamanho mínimo
max_size = 20  # tamanho máximo

min_betweenness = betweenness_values.min()
max_betweenness = betweenness_values.max()

# Criar uma propriedade de vértice para os tamanhos baseados em betweenness
vertex_size_betweenness = g.new_vp("double")
for v in g.vertices():
    betweenness_val = vertex_betweenness[v]
    # Normalização linear dos valores para o intervalo [min_size, max_size]
    if max_betweenness > min_betweenness:
        normalized = (betweenness_val - min_betweenness) / (max_betweenness - min_betweenness)
    else:
        normalized = 0
    vertex_size_betweenness[v] = min_size + normalized * (max_size - min_size)

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Gerar o diagrama, definindo o tamanho dos vértices conforme o betweenness
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_betweenness,
    output_size=(1000, 1000),
    output="graph_betweenness.png"
)

print("Diagrama gerado: 'graph_betweenness.png'")

Diagrama gerado: 'graph_betweenness.png'


KeyboardInterrupt: 

In [23]:
# Versão simplificada e mais eficiente para calcular uma métrica de restrição estrutural
# Em vez de calcular a restrição completa de Burt, vamos usar uma medida relacionada
# chamada "coeficiente de agrupamento local" (local clustering coefficient)
from graph_tool.clustering import local_clustering

# Calcular o coeficiente de agrupamento local (relacionado à restrição)
clustering_prop = local_clustering(g)

# Extrair os valores de clustering para normalização
clustering_values = np.array([clustering_prop[v] for v in g.vertices()])

# Definir tamanho mínimo e máximo para os vértices
min_size = 5   # tamanho mínimo
max_size = 20  # tamanho máximo

# Lidar com possíveis NaN (Not a Number) resultados de divisão por zero
clustering_values = np.nan_to_num(clustering_values)

min_clustering = clustering_values.min()
max_clustering = clustering_values.max()

# Criar uma propriedade de vértice para os tamanhos
# O clustering é proporcional à restrição - maior clustering = maior restrição
vertex_size_clustering = g.new_vp("double")
for v in g.vertices():
    clustering_val = clustering_prop[v]
    if np.isnan(clustering_val):
        clustering_val = 0
    
    # Invertemos para que menor clustering (≈ menor restrição) = maior nó
    if max_clustering > min_clustering:
        normalized = 1 - ((clustering_val - min_clustering) / (max_clustering - min_clustering))
    else:
        normalized = 0
    
    vertex_size_clustering[v] = min_size + normalized * (max_size - min_size)

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Gerar o diagrama
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_clustering,
    output_size=(1000, 1000),
    output="graph_structural_holes.png"
)

print("Diagrama gerado: 'graph_structural_holes.png'")

Diagrama gerado: 'graph_structural_holes.png'


In [24]:
from graph_tool.topology import kcore_decomposition

# Calcular a decomposição k-core
# Esta é uma boa aproximação para uma medida contínua de centro-periferia
kcore = kcore_decomposition(g)

# Extrair os valores de k-core para normalização
kcore_values = np.array([kcore[v] for v in g.vertices()])

# Definir tamanho mínimo e máximo para os vértices
min_size = 5   # tamanho mínimo
max_size = 20  # tamanho máximo

min_kcore = kcore_values.min()
max_kcore = kcore_values.max()

# Criar uma propriedade de vértice para os tamanhos baseados no k-core
# Valores maiores de k-core indicam posição mais central
vertex_size_coreness = g.new_vp("double")
for v in g.vertices():
    kcore_val = kcore[v]
    # Normalização linear dos valores para o intervalo [min_size, max_size]
    if max_kcore > min_kcore:
        normalized = (kcore_val - min_kcore) / (max_kcore - min_kcore)
    else:
        normalized = 0
    vertex_size_coreness[v] = min_size + normalized * (max_size - min_size)

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Gerar o diagrama, definindo o tamanho dos vértices conforme o k-core
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_coreness,
    output_size=(1000, 1000),
    output="graph_core_periphery.png"
)

print("Diagrama gerado: 'graph_core_periphery.png'")

Diagrama gerado: 'graph_core_periphery.png'


In [25]:
from graph_tool.topology import kcore_decomposition
import numpy as np

# Usar k-core decomposition como base para identificar centro-periferia
kcore = kcore_decomposition(g)

# Extrair os valores de k-core
kcore_values = np.array([kcore[v] for v in g.vertices()])

# Encontrar um threshold para dividir discretamente entre centro e periferia
# Podemos usar a mediana ou um percentil específico
kcore_threshold = np.percentile(kcore_values, 75)  # Usando o percentil 75 (top 25% = centro)

# Criar uma propriedade discreta de centro-periferia
# 1 = centro, 0 = periferia
core_periphery = g.new_vp("int")
for v in g.vertices():
    if kcore[v] >= kcore_threshold:
        core_periphery[v] = 1  # Centro
    else:
        core_periphery[v] = 0  # Periferia

# Definir tamanhos para centro e periferia
center_size = 15    # Tamanho para nós do centro
periphery_size = 5  # Tamanho para nós da periferia

# Criar propriedade de tamanho baseada na classificação discreta
vertex_size_discrete = g.new_vp("double")
for v in g.vertices():
    if core_periphery[v] == 1:
        vertex_size_discrete[v] = center_size
    else:
        vertex_size_discrete[v] = periphery_size

# Posicionar o grafo utilizando o layout SFDP
layout = gte.sfdp_layout(g)

# Também vamos colorir os nós de acordo com sua classificação
vertex_color = g.new_vp("vector<double>")
for v in g.vertices():
    if core_periphery[v] == 1:
        vertex_color[v] = [0.2, 0.4, 0.8, 0.8]  # Azul para centro
    else:
        vertex_color[v] = [0.8, 0.4, 0.2, 0.8]  # Laranja para periferia

# Gerar o diagrama com tamanhos e cores discretos
gte.graph_draw(
    g,
    pos=layout,
    vertex_size=vertex_size_discrete,
    vertex_fill_color=vertex_color,
    output_size=(1000, 1000),
    output="graph_discrete_core_periphery.png"
)

# Imprimir estatísticas sobre a classificação
num_core = sum(1 for v in g.vertices() if core_periphery[v] == 1)
num_periphery = g.num_vertices() - num_core
print(f"Diagrama gerado: 'graph_discrete_core_periphery.png'")
print(f"Classificação centro-periferia:")
print(f"- Nós no centro: {num_core} ({(num_core/g.num_vertices())*100:.1f}%)")
print(f"- Nós na periferia: {num_periphery} ({(num_periphery/g.num_vertices())*100:.1f}%)")

Diagrama gerado: 'graph_discrete_core_periphery.png'
Classificação centro-periferia:
- Nós no centro: 54 (27.1%)
- Nós na periferia: 145 (72.9%)


In [26]:
import matplotlib.pyplot as plt
import numpy as np

# Calcular in-degree, out-degree e total degree para cada vértice
in_degrees = np.array([v.in_degree() for v in g.vertices()])
out_degrees = np.array([v.out_degree() for v in g.vertices()])
total_degrees = in_degrees + out_degrees

# Criar figura com 3 subplots para diferentes tipos de degree
fig, axs = plt.subplots(3, 1, figsize=(10, 15))

# 1. Histograma regular da distribuição de degree
axs[0].hist(total_degrees, bins=30, alpha=0.7, color='blue')
axs[0].set_title('Distribuição de Degree (Total)')
axs[0].set_xlabel('Degree (Total)')
axs[0].set_ylabel('Número de Países')
axs[0].grid(True, alpha=0.3)

# 2. Histograma em escala log-log (útil para identificar power law)
# Remover zeros antes de aplicar log
nonzero_degrees = total_degrees[total_degrees > 0]

if len(nonzero_degrees) > 0:
    # Criar bins em escala logarítmica
    log_bins = np.logspace(np.log10(nonzero_degrees.min()), 
                         np.log10(nonzero_degrees.max()), 
                         20)
    
    # Plotar histograma com escala log-log
    axs[1].hist(nonzero_degrees, bins=log_bins, alpha=0.7, color='green')
    axs[1].set_xscale('log')
    axs[1].set_yscale('log')
    axs[1].set_title('Distribuição de Degree (Log-Log)')
    axs[1].set_xlabel('Log(Degree)')
    axs[1].set_ylabel('Log(Frequência)')
    axs[1].grid(True, alpha=0.3)

# 3. Gráfico complementar de distribuição cumulativa (CCDF)
# Frequentemente usado para analisar distribuições power-law
sorted_degrees = np.sort(total_degrees)
ccdf = 1 - np.arange(1, len(sorted_degrees) + 1) / len(sorted_degrees)

axs[2].scatter(sorted_degrees, ccdf, alpha=0.7, color='red')
axs[2].set_xscale('log')
axs[2].set_yscale('log')
axs[2].set_title('Distribuição Cumulativa Complementar (CCDF)')
axs[2].set_xlabel('Degree (k)')
axs[2].set_ylabel('P(K ≥ k)')
axs[2].grid(True, alpha=0.3)

# Ajustar layout e salvar
plt.tight_layout()
plt.savefig('degree_distribution.png', dpi=300)
plt.close()

print("Distribuição de degree salva como 'degree_distribution.png'")

# Estatísticas adicionais
mean_degree = np.mean(total_degrees)
median_degree = np.median(total_degrees)
max_degree = np.max(total_degrees)
min_degree = np.min(total_degrees)

print(f"\nEstatísticas da distribuição de degree:")
print(f"- Degree médio: {mean_degree:.2f}")
print(f"- Degree mediano: {median_degree:.2f}")
print(f"- Degree máximo: {max_degree}")
print(f"- Degree mínimo: {min_degree}")

# Se houver evidência de power law, verificar a inclinação (alfa)
if len(nonzero_degrees) > 0:
    log_degrees = np.log10(nonzero_degrees)
    log_freq, _ = np.histogram(log_degrees, bins=20)
    # Filtrar para remover frequências zeradas
    nonzero_indices = log_freq > 0
    if np.sum(nonzero_indices) > 2:  # Precisamos de pelo menos 3 pontos
        from scipy import stats
        # Calcular a inclinação da linha no gráfico log-log
        bin_centers = (_[:-1] + _[1:]) / 2
        bin_centers = bin_centers[nonzero_indices]
        log_freq = log_freq[nonzero_indices]
        log_freq = np.log10(log_freq)
        
        slope, intercept, r_value, p_value, std_err = stats.linregress(bin_centers, log_freq)
        print(f"- Inclinação da distribuição em log-log: {slope:.2f}")
        print(f"- R² da regressão linear: {r_value**2:.2f}")
        
        if r_value**2 > 0.8 and slope < 0:
            print("- Há evidência de distribuição power-law (cauda longa)")
            if slope < -2:
                print("- Inclinação fortemente negativa sugere estrutura centro-periferia bem definida")

Distribuição de degree salva como 'degree_distribution.png'

Estatísticas da distribuição de degree:
- Degree médio: 117.46
- Degree mediano: 91.00
- Degree máximo: 368
- Degree mínimo: 5
- Inclinação da distribuição em log-log: 0.57
- R² da regressão linear: 0.51
